In [1]:
#- Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요. 체인을 수동으로 구현해야 합니다.
#- 체인에 ConversationBufferMemory를 부여합니다.
#- 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
#- 체인에 다음 질문을 합니다:
#	Aaronson 은 유죄인가요?
#	그가 테이블에 어떤 메시지를 썼나요?
#	Julia 는 누구인가요?

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI( temperature=0.3, streaming=True ) 

memory = ConversationBufferMemory(llm=llm, max_token_limit=20, return_messages=True) # 메모리 설정

cache_dir = LocalFileStore("./.cache/") # 캐싱 저장경로 설정

loader = UnstructuredFileLoader("./files/assignment4.txt") # 문서 load

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
) # splitter 설정

splitted_docs = loader.load_and_split(text_splitter=splitter) # 문서 객체를 split

embeddings = OpenAIEmbeddings() # 오픈ai 임베딩 모델 사용

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir) # 임베딩을 디렉토리에 캐싱

vectorstore = FAISS.from_documents(splitted_docs, cached_embeddings) # FAISS 벡터스토어 생성 & 문서 저장

retriever = vectorstore.as_retriever() # retriever 구현



/tmp/ipykernel_1491/1334407323.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI( temperature=0.3, streaming=True )
/tmp/ipykernel_1491/1334407323.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(llm=llm, max_token_limit=20, return_messages=True) # 메모리 설정
/tmp/ipykernel_1491/1334407323.py:17: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip insta

In [5]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{document}"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}")]
)

chain = (
    {
        "document": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
) | prompt | llm

def invoke_chain(question):
    #print(type(question))
    result = chain.invoke(question)
    memory.save_context(
        {"input":question}, {"output":result.content}
    )
    print(result)

invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

load_memory({})

AttributeError: 'ConversationBufferMemory' object has no attribute 'save_content'